In [ ]:
# Import necessary libraries
import csv
from datetime import datetime, timedelta
import sqlite3
from tabulate import tabulate

# Define the Book class
class Book:
    def __init__(self, title, author, isbn, quantity):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.quantity = quantity

    def display_details(self):
        print(f"Title: {self.title}")
        print(f"Author: {self.author}")
        print(f"ISBN: {self.isbn}")
        print(f"Quantity: {self.quantity}")

# Define the Patron class
class Patron:
    def __init__(self, name, id, contact_info):
        self.name = name
        self.id = id
        self.contact_info = contact_info

    def display_details(self):
        print(f"Name: {self.name}")
        print(f"ID: {self.id}")
        print(f"Contact Info: {self.contact_info}")

# Define the Transaction class
class Transaction:
    def __init__(self, book_id, patron_id, checkout_date, due_date):
        self.book_id = book_id
        self.patron_id = patron_id
        self.checkout_date = checkout_date
        self.due_date = due_date

class Librarian:
    def __init__(self, name, employee_id):
        self.name = name
        self.employee_id = employee_id

    def add_book(self, library, book):
        library.add_book(book)

    def update_book_quantity(self, library, book_id, new_quantity):
        library.update_book(book_id, new_quantity)

    def remove_book(self, library, book_id):
        library.remove_book(book_id)

    def add_patron(self, library, patron):
        library.add_patron(patron)

    def update_patron_contact_info(self, library, patron_id, new_contact_info):
        library.update_patron(patron_id, new_contact_info)

class Admin(Librarian):
    # Admin inherits Librarian methods and attributes
    def __init__(self, name, employee_id):
        super().__init__(name, employee_id)

    def add_librarian(self, library, librarian):
        library.librarians.append(librarian)

    def remove_librarian(self, library, librarian):
        library.librarians.remove(librarian)

    def generate_report_all(self, library):
        library.generate_report()

if __name__ == "__main__":
    librarian = Librarian("Alice", "L001")
    admin = Admin("Bob", "A001")

    # Use the librarian and admin objects to perform actions in the library management system

# Define the Library class
class Library:

    def __init__(self):
            # Define SQLite connection and cursor
        self.conn = sqlite3.connect('library.db')
        self.cursor = self.conn.cursor()
        self.books = []
        self.patrons = []
        self.transactions = []
        # Create a Books table
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS Books (
                    isbn TEXT PRIMARY KEY,
                    title TEXT,
                    author TEXT,
                    quantity INTEGER
                )''')

    def add_fine(self, patron_id, amount):
        self.cursor.execute("INSERT INTO Fines (patron_id, amount) VALUES (?, ?)", (patron_id, amount))
        self.conn.commit()

    def pay_fine(self, patron_id, amount):
        self.cursor.execute("SELECT amount FROM Fines WHERE patron_id = ?", (patron_id,))
        current_amount = self.cursor.fetchone()
        if current_amount:
            new_amount = max(current_amount[0] - amount, 0)
            self.cursor.execute("UPDATE Fines SET amount = ? WHERE patron_id = ?", (new_amount, patron_id))
            self.conn.commit()
        else:
            print("Patron does not have any fines.")

    def generate_report_fines(self):
        print("\nFines:")
        self.cursor.execute("SELECT * FROM Fines")
        fines = self.cursor.fetchall()
        for i in fines:
            print(f"Patron ID: {i[1]}, Amount: {i[2]}")

# Create a Patrons table
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS Patrons (
                    id TEXT PRIMARY KEY,
                    name TEXT,
                    contact_info TEXT
                )''')

# Create a Transactions table
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS Transactions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    book_id TEXT,
                    patron_id TEXT,
                    checkout_date TEXT,
                    due_date TEXT
                )''')

    def add_book(self, book):
        self.books.append(book)

    def update_book(self, book_id, new_quantity):
        for book in self.books:
            if book.isbn == book_id:
                book.quantity = new_quantity
                break

    def remove_book(self, book_id):
        for book in self.books:
            if book.isbn == book_id:
                self.books.remove(book)
                break

    def add_patron(self, patron):
        self.patrons.append(patron)

    def update_patron(self, patron_id, new_contact_info):
        for patron in self.patrons:
            if patron.id == patron_id:
                patron.contact_info = new_contact_info
                break

    def remove_patron(self, patron_id):
        for patron in self.patrons:
            if patron.id == patron_id:
                self.patrons.remove(patron)
                break

    def checkout_book(self, book_id, patron_id):
        # Check if the book is available
        book_available = False
        for book in self.books:
            if book.isbn == book_id and book.quantity > 0:
                book_available = True
                break

        # If the book is available, create a new transaction
        if book_available:
            checkout_date = datetime.today()
            due_date = checkout_date + timedelta(days=14)
            new_transaction = Transaction(book_id, patron_id, checkout_date, due_date)
            self.transactions.append(new_transaction)
            for book in self.books:
                if book.isbn == book_id:
                    book.quantity -= 1
        else:
            print("Book not available for checkout.")

    def return_book(self, book_id, patron_id):
        # Find the transaction associated with the book and patron
        for transaction in self.transactions:
            if transaction.book_id == book_id and transaction.patron_id == patron_id:
                for book in self.books:
                    if book.isbn == book_id:
                        book.quantity += 1
                self.transactions.remove(transaction)
                break
        else:
            print("Transaction not found.")

    def generate_report(self):
        # Generate a report of all books, patrons, and transactions
        book_data = [[book.title, book.author, book.isbn, book.quantity] for book in self.books]
        patron_data = [[patron.name, patron.id, patron.contact_info] for patron in self.patrons]
        transaction_data = [[transaction.book_id, transaction.patron_id, transaction.checkout_date, transaction.due_date] for transaction in self.transactions]

        print("Books:")
        print(tabulate(book_data, headers=["Title", "Author", "ISBN", "Quantity"], tablefmt="grid"))

        print("\nPatrons:")
        print(tabulate(patron_data, headers=["Name", "ID", "Contact Info"], tablefmt="grid"))

        print("\nTransactions:")
        print(tabulate(transaction_data, headers=["Book ID", "Patron ID", "Checkout Date", "Due Date"], tablefmt="grid"))
# lib2
    def close_db_connection(self):
        self.conn.commit()
        self.conn.close()



# Test the code
if __name__ == "__main__":
    library = Library()


    Book: book1
    Book: book2
    Book: book3

    book1 = Book("Introduction to Python", "Mike Driscoll", "123456789", 5)
    book2 = Book("Machine Learning for Dummies", "Peter Harrington", "987654321", 3)
    book3 = Book("The Hitchhiker's Guide to the Galaxy", "Douglas Adams", "000000001", 2)

    Patron: patron1
    Patron: patron2
    Patron: patron3

    patron1 = Patron("Sally Johnson", "0001", "sally@example.com")
    patron2 = Patron("Bill Brown", "0002", "bill@example.com")
    patron3 = Patron("Ted Williams", "0003", "ted@example.com")

    library.add_book(book1)
    library.add_book(book2)
    library.add_book(book3)

    library.add_patron(patron1)
    library.add_patron(patron2)
    library.add_patron(patron3)

    library.generate_report()

    # Perform some actions on the library objects
    library.update_book("123456789", 10)
    library.return_book("123456789", "0001")
    library.checkout_book("987654321", "0002")
    library.remove_book("000000001")
    library.remove_patron("0003")

    library.generate_report()



# Implement CLI for User Interface

def print_menu():
    print("Library Management System - Main Menu")
    print("1. Add Book")
    print("2. Add Patron")
    print("3. Checkout Book")
    print("4. Return Book")
    print("5. Generate Report")
    print("6. Exit")
    return input("Enter your choice: ")

def main():
    library = Library()

    while True:
        choice = print_menu()

        if choice == '1':
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            isbn = input("Enter book ISBN: ")
            quantity = int(input("Enter quantity: "))

            book = Book(title, author, isbn, quantity)
            library.add_book(book)

        elif choice == '2':
            name = input("Enter patron name: ")
            id = input("Enter patron ID: ")
            contact_info = input("Enter patron contact info: ")

            patron = Patron(name, id, contact_info)
            library.add_patron(patron)

        elif choice == '3':
            isbn = input("Enter book ISBN to checkout: ")
            patron_id = input("Enter patron ID: ")

            found_book = False
            for book in library.books:
                if book.isbn == isbn and book.quantity > 0:
                    found_book = True
                    break

            if found_book:
                library.checkout_book(isbn, patron_id)
                print("Book checked out successfully!")
            else:
                print("Book not available for checkout.")

        elif choice == '4':
            isbn = input("Enter book ISBN to return: ")
            patron_id = input("Enter patron ID: ")

            found_transaction = False
            for transaction in library.transactions:
                if transaction.book_id == isbn and transaction.patron_id == patron_id:
                    found_transaction = True
                    break

            if found_transaction:
                library.return_book(isbn, patron_id)
                print("Book returned successfully!")
            else:
                print("Transaction not found.")

        elif choice == '5':
            library.generate_report()

        elif choice == '6':
            print("Exiting Library Management System.")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()

Books:
+--------------------------------------+------------------+-----------+------------+
| Title                                | Author           |      ISBN |   Quantity |
+======================================+==================+===========+============+
| Introduction to Python               | Mike Driscoll    | 123456789 |          5 |
+--------------------------------------+------------------+-----------+------------+
| Machine Learning for Dummies         | Peter Harrington | 987654321 |          3 |
+--------------------------------------+------------------+-----------+------------+
| The Hitchhiker's Guide to the Galaxy | Douglas Adams    | 000000001 |          2 |
+--------------------------------------+------------------+-----------+------------+

Patrons:
+---------------+------+-------------------+
| Name          |   ID | Contact Info      |
+===============+======+===================+
| Sally Johnson | 0001 | sally@example.com |
+---------------+------+--------------